In [1]:
import multiprocess
import urllib
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
# Read, parse, and organize the data
df = pd.read_csv('WikiArt-Emotions-All.tsv', sep='\t')
obsolete_columns = [column for column in df.columns
                    if column != 'Image URL' and
                    column != 'ID' and
                    'ImageOnly' not in column]
df = df.drop(columns=obsolete_columns)
df = df.rename(columns={name: name.split(' ')[1] for name in df.columns
                        if name != 'Image URL' and
                        name != 'ID'})
df = df.set_index('ID')

# Save the dataframes for later use
df.to_pickle('data.pkl')

In [ ]:
# Create a downloading function we can multithread
def download_image(iterrow):
    index, row = iterrow
    url = row['Image URL'].replace('https', 'http')
    urllib.urlretrieve(url, 'images/' + index + '.jpg')


# Download the training images
pool = multiprocess.Pool(4)
iterator = pool.imap(download_image, df.iterrows())
_ = list(tqdm_notebook(iterator, total=df.shape[0]))